In [2]:
##  Загрузка библиотек
import pandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge, LinearRegression
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

In [3]:
## Читаем дф 
df = pandas.read_csv('gamesPC.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16598 non-null  int64  
 1   Name          16598 non-null  object 
 2   Platform      16598 non-null  object 
 3   Year          16327 non-null  float64
 4   Genre         16598 non-null  object 
 5   Publisher     16540 non-null  object 
 6   NA_Sales      16598 non-null  float64
 7   EU_Sales      16598 non-null  float64
 8   JP_Sales      16598 non-null  float64
 9   Other_Sales   16598 non-null  float64
 10  Global_Sales  16598 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB


In [5]:
df.head(5)

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [6]:
##  Кодировка в числа
categorical_columns = df.select_dtypes(include='object').columns
encoder = preprocessing.LabelEncoder()

le = LabelEncoder()
df['Name'] = encoder.fit_transform(df['Name'])
df['Platform'] = encoder.fit_transform(df['Platform'])
df['Genre'] = encoder.fit_transform(df['Genre'])
df['Publisher'] = df['Publisher'].apply(str)
df['Publisher'] = encoder.fit_transform(df['Publisher'])
df.head(5)

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,11007,26,2006.0,10,359,41.49,29.02,3.77,8.46,82.74
1,2,9327,11,1985.0,4,359,29.08,3.58,6.81,0.77,40.24
2,3,5573,26,2008.0,6,359,15.85,12.88,3.79,3.31,35.82
3,4,11009,26,2009.0,10,359,15.75,11.01,3.28,2.96,33.00
4,5,7346,5,1996.0,7,359,11.27,8.89,10.22,1.00,31.37


In [7]:
##  иницилизируем df1 для прогноза
df1 = df
df1.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,11007,26,2006.0,10,359,41.49,29.02,3.77,8.46,82.74
1,2,9327,11,1985.0,4,359,29.08,3.58,6.81,0.77,40.24
2,3,5573,26,2008.0,6,359,15.85,12.88,3.79,3.31,35.82
3,4,11009,26,2009.0,10,359,15.75,11.01,3.28,2.96,33.00
4,5,7346,5,1996.0,7,359,11.27,8.89,10.22,1.00,31.37


In [8]:
##  Избавляемся от NaN и заполняем средними
df1['Year'] = df['Year'].astype(float)
df1 = df1.fillna(0) ##Избавляется
df1['Year'].replace(0,df1['Year'].mean(axis=0),inplace=True) ##Заполняет
##
#df1.replace(0,df1.mean(axis=0),inplace=True)
#df1.fillna(df1.mean())
##
df1['Year'] = df1.Year.astype(int) ## переводим в int 
##
#df1['Year'].value_counts()
df1.head(5)
#df1.iloc[0:3]


,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,11007,26,2006,10,359,41.49,29.02,3.77,8.46,82.74
1,2,9327,11,1985,4,359,29.08,3.58,6.81,0.77,40.24
2,3,5573,26,2008,6,359,15.85,12.88,3.79,3.31,35.82
3,4,11009,26,2009,10,359,15.75,11.01,3.28,2.96,33.00
4,5,7346,5,1996,7,359,11.27,8.89,10.22,1.00,31.37


In [9]:
df1['Year'].mean()
#print(df1.loc['0'].values)
df1

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,11007,26,2006,10,359,41.49,29.02,3.77,8.46,82.74
1,2,9327,11,1985,4,359,29.08,3.58,6.81,0.77,40.24
2,3,5573,26,2008,6,359,15.85,12.88,3.79,3.31,35.82
3,4,11009,26,2009,10,359,15.75,11.01,3.28,2.96,33.00
4,5,7346,5,1996,7,359,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,11101,6,2002,4,269,0.01,0.00,0.00,0.00,0.01
16594,16597,5796,7,2003,8,241,0.01,0.00,0.00,0.00,0.01
16595,16598,8144,16,2008,6,21,0.00,0.00,0.00,0.00,0.01
16596,16599,5014,4,2010,5,8,0.00,0.01,0.00,0.00,0.01


In [10]:
## Шафулим
from sklearn.utils import shuffle  
df1 = shuffle(df1) 
df1

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
8470,8472,9109,16,2001,6,288,0.08,0.06,0.00,0.02,0.16
9707,9709,6196,4,2010,7,347,0.00,0.00,0.12,0.00,0.12
13147,13149,4831,23,1993,10,332,0.00,0.00,0.05,0.00,0.05
12276,12278,3788,16,2000,9,275,0.00,0.00,0.06,0.00,0.06
15027,15030,4855,15,1997,2,275,0.01,0.01,0.00,0.00,0.02
...,...,...,...,...,...,...,...,...,...,...,...
10510,10512,1982,16,2005,0,456,0.05,0.04,0.00,0.01,0.10
8059,8061,5199,26,2008,3,446,0.09,0.06,0.01,0.02,0.18
1343,1345,1951,17,2011,8,138,0.73,0.50,0.00,0.21,1.44
10933,10935,9096,13,2007,11,288,0.01,0.06,0.00,0.01,0.09


In [11]:
#df1 = df1.drop('index',1)
## Тренируем модель
forest = RandomForestRegressor()
X = df1.values[:, 0:6]
Y = df1.values[:, 6]
trainX, testX, trainY, testY = train_test_split( X, Y, test_size = 0.3)
forest.fit(trainX, trainY)
print('Accuracy: \n', forest.score(testX, testY))
pred = forest.predict(testX)

Accuracy: 
 0.7886089182098447


In [12]:
#import matplotlib.pyplot as plt
#import numpy as np
#ind = np.arange(df1['Rank'].count())
#plt.bar(ind, df1['Rank'])
#plt.show()

In [13]:
## Сохраняем и загружем модель
import joblib
# create RF
rf = RandomForestRegressor()
# fit on some data
rf.fit(trainX, trainY)

# save
joblib.dump(rf, "my_random_forest.joblib")
    
# load
loaded_rf = joblib.load("my_random_forest.joblib")

In [14]:
## Чё то делаем с моделью, проверям работу
#loaded_model = joblib.load(filename)
result = loaded_rf.score(testX, testY)
print(result)
df1

0.7765878024668713


,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
8470,8472,9109,16,2001,6,288,0.08,0.06,0.00,0.02,0.16
9707,9709,6196,4,2010,7,347,0.00,0.00,0.12,0.00,0.12
13147,13149,4831,23,1993,10,332,0.00,0.00,0.05,0.00,0.05
12276,12278,3788,16,2000,9,275,0.00,0.00,0.06,0.00,0.06
15027,15030,4855,15,1997,2,275,0.01,0.01,0.00,0.00,0.02
...,...,...,...,...,...,...,...,...,...,...,...
10510,10512,1982,16,2005,0,456,0.05,0.04,0.00,0.01,0.10
8059,8061,5199,26,2008,3,446,0.09,0.06,0.01,0.02,0.18
1343,1345,1951,17,2011,8,138,0.73,0.50,0.00,0.21,1.44
10933,10935,9096,13,2007,11,288,0.01,0.06,0.00,0.01,0.09


In [15]:
#from keras.models import load_model

#model = RandomForestRegressor() # Train your model

#model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
#model = load_model('my_model.h5') # Load your saved model and use it on whatever data you want

In [16]:
## Тестирую мою модель
a = [[10805,1301,2,2011,2,120]]
loaded_rf.predict(a)
#df1

array([0.064])

In [17]:
## DF для тестов введённого массива

#Читаем df
df2 = pandas.read_csv('gamesPC.csv')
#Выкидываем ненужное
df2.drop(['EU_Sales', 'JP_Sales','Other_Sales','Global_Sales','NA_Sales'], axis=1, inplace=True)
# Создаю массив
a1 = 1
a2 = 'Wii Sports'
a3 = 'Wii'
a4 = 2006
a5 = 'Sports'
a6 = 'Nintendo'
###
#array = [[a1,a2,a3,a4,a5,a6]]

#df2.index(6540) = [11434,'Pokemon Red/Pokemon Blue','GB','1996','Role-Playing','Nintendo']

#df2.loc[len(df2)] = [11434, 'Pokemon Red/Pokemon Blue', 'GB', 1996, 'Role-Playing', 'Nintendo']

#df2.loc[len(df2)] = [[a1,a2,a3,a4,a5,a6]]

array = [a1,a2,a3,a4,a5,a6]
###
# Суём в последнюю строку  DF'a
df2.loc['16598'] = array
#
#row = ['a3','b3', np.nan]
#newDF.loc['2'] = row
#
# Кодируем в DF
df2['Name'] = encoder.fit_transform(df2['Name'])
df2['Platform'] = encoder.fit_transform(df2['Platform'])
df2['Genre'] = encoder.fit_transform(df2['Genre'])
df2['Publisher'] = df2['Publisher'].apply(str) 
df2['Publisher'] = encoder.fit_transform(df2['Publisher'])
# Переводим ссаную строку в STR, хз зачем - просто так работает хотя бы
df2['Year'] = df2['Year'].apply(str)

print(df2.loc['16598'].values)
df2

[1 11007 26 '2006.0' 10 359]


,Rank,Name,Platform,Year,Genre,Publisher
0,1,11007,26,2006.0,10,359
1,2,9327,11,1985.0,4,359
2,3,5573,26,2008.0,6,359
3,4,11009,26,2009.0,10,359
4,5,7346,5,1996.0,7,359
...,...,...,...,...,...,...
16594,16597,5796,7,2003.0,8,241
16595,16598,8144,16,2008.0,6,21
16596,16599,5014,4,2010.0,5,8
16597,16600,8967,6,2003.0,4,546


In [18]:
# Переводим во флоат, Причина та же ,что сверху 
df2['Year'] = df2['Year'].astype(float)
#
#df2 = df2.fillna(0)
#df2['Year'].replace(0,df2['Year'].mean(axis=0),inplace=True)
#df2.replace(0,df2.mean(axis=0),inplace=True)
#df2.fillna(df2.mean())

#df2.fillna(df2.mean())
#df2['Year'] = df2.Year.astype(int)

#df2.apply(lambda x: x.fillna(x.mean()),axis=0)
#
# Заполняем пустые значения - средними
df2['Year'].fillna((df2['Year'].mean()), inplace=True)
# Переводим в Int
df2['Year'] = df2.Year.astype(int)
df2

,Rank,Name,Platform,Year,Genre,Publisher
0,1,11007,26,2006,10,359
1,2,9327,11,1985,4,359
2,3,5573,26,2008,6,359
3,4,11009,26,2009,10,359
4,5,7346,5,1996,7,359
...,...,...,...,...,...,...
16594,16597,5796,7,2003,8,241
16595,16598,8144,16,2008,6,21
16596,16599,5014,4,2010,5,8
16597,16600,8967,6,2003,4,546


In [19]:
#df2['Year'] = df2['Year'].apply(float)
df2.dtypes
#print(df2.loc[16598].values)

Rank         int64
Name         int32
Platform     int32
Year         int32
Genre        int32
Publisher    int32
dtype: object

In [20]:
#print(df2.loc[16598].values)

In [21]:
## Вытаскиваю нужные данные и сую в массив
b = df2.loc['16598'].values
b

array([    1, 11007,    26,  2006,    10,   359], dtype=int64)

In [22]:
## Проверяю результат прогноза с результатом из DF
#loaded_rf.predict([b])

b = [10660,2141,17,2014,2,347]
b = [173,492,28,2012,0,525]
loaded_rf.predict([b])


array([2.9535])

In [23]:
df3 = pandas.read_csv('gamesPC.csv')

In [24]:
df3.iloc[3]

Rank                            4
Name            Wii Sports Resort
Platform                      Wii
Year                         2009
Genre                      Sports
Publisher                Nintendo
NA_Sales                    15.75
EU_Sales                    11.01
JP_Sales                     3.28
Other_Sales                  2.96
Global_Sales                   33
Name: 3, dtype: object

In [25]:
array = df3.iloc[3]
    
df4 = pandas.read_csv('gamesPC.csv')
df4.drop(['NA_Sales','EU_Sales', 'JP_Sales','Other_Sales','Global_Sales'], axis=1, inplace=True)
    
df4.loc['16598'] = array

categorical_columns = df4.select_dtypes(include='object').columns
encoder = preprocessing.LabelEncoder()

le = LabelEncoder()

df4['Name'] = encoder.fit_transform(df4['Name'])
df4['Platform'] = encoder.fit_transform(df4['Platform'])
df4['Genre'] = encoder.fit_transform(df4['Genre'])
df4['Publisher'] = df4['Publisher'].apply(str) 
df4['Publisher'] = encoder.fit_transform(df4['Publisher'])

df4['Year'] = df4['Year'].apply(str)
df4['Year'] = df4['Year'].astype(float)

# Заполняем пустые значения - средними
df4['Year'].fillna((df4['Year'].mean()), inplace=True)
# Переводим в Int
df4['Year'] = df4.Year.astype(int)

# load
loaded_rf = joblib.load("my_random_forest.joblib")

array = df4.iloc[3]

print(loaded_rf.predict([array]))
array

[14.8681]


Rank             4
Name         11009
Platform        26
Year          2009
Genre           10
Publisher      359
Name: 3, dtype: int64

In [26]:
x = None
if x:
    print('Не пустое')
else: print('Пустое')

Пустое


In [29]:
df4['Year'].max()

2020

In [30]:
df4['Year'].min()

1980

In [31]:
df4['Year'].mean()

2006.3999638532441